In [1]:
# from time import sleep
# from random import randint
from bs4 import BeautifulSoup
# import datetime
import requests
import pandas as pd
import re

In [2]:
def get_page_source(url):
    agent = {"User-Agent":"Mozilla/5.0"} # User Agent defined to avoid robot detection
    source=requests.get(url, headers=agent)
    return source.text, source.status_code

def get_soup(content):
    soup=BeautifulSoup(content, "lxml")
    return soup

In [3]:
def get_global_orgs(url = "http://geni.org/globalenergy/library/organizations/#global"):
    text, status = get_page_source(url)
    soup = get_soup(text)
    ul_all = soup.findAll('ul')
    desired_ul = [ul_all[3], ul_all[4], ul_all[5] ] # 4th, 5th, 6th ul contains data
    result = list()
    region = "Global"
    li_all = desired_ul[0].findAll('li')
        
    for li in li_all:
        if li.find('h2'): # Change region
            region = li.find('h2').text.strip()
        title = li.find('b').text.strip()
        title = title.replace('\n', '')
        link = li.find('a').get('href')
        email = ""
        href_all = li.findAll('a')
        if(len(href_all) > 1): # Some emails are missing
            email = li.findAll('a')[1].text.strip()
        result.append({'Title' : title,
                      'Website' : link,
                      'Email' : email,
                      'Region' : region})
    
    li_all = desired_ul[1].findAll('li')
    region = "Asia"
    for li in li_all:
        if(li.next_sibling):
            if(li.next_sibling.find(attrs={"name" : "middleeastafrica"})): # Change region
                region = "Middle East/Africa"
        title = li.find('b').text.strip()
        title = title.replace('\n', '')
        link = li.find('a').get('href')
        email = ""
        href_all = li.findAll('a')
        if(len(href_all) > 1): # Some emails are missing
            email = li.findAll('a')[1].text.strip()
        result.append({'Title' : title,
                      'Website' : link,
                      'Email' : email,
                      'Region' : region})
    
    
    region = "Middle East/Africa"
    li_all = desired_ul[2].findAll('li')
    for li in li_all:
        title = li.find('b').text.strip()
        title = title.replace('\n', '')
        link = li.find('a').get('href')
        email = ""
        href_all = li.findAll('a')
        if(len(href_all) > 1): # Some emails are missing
            email = li.findAll('a')[1].text.strip()
        result.append({'Title' : title,
                      'Website' : link,
                      'Email' : email,
                      'Region' : region})

            
    
   
    return pd.DataFrame(result)
    

In [4]:
df = get_global_orgs()

In [6]:
df.to_csv("GENI.csv")
df

,Title,Website,Email,Region
0,European Bank for Reconstruction and Development,http://www.ebrd.com/,projectenquiries@ebrd.com,Global
1,European Union,http://europa.eu.int/,envinfo@cec.eu.int,Global
2,Greenpeace International,http://www.greenpeace.org/,supporter.services@ams.greenpeace.org,Global
3,International Finance Corporation,http://www.ifc.org/,(IFC),Global
4,International Renewable Energy Agency,http://www.irena.org/,irenapress@irena.org,Global
...,...,...,...,...
63,Swisscontact in Eastern and Sothern Africa,http://www.swisscontact.org/,sc@swisscontact.ch,Middle East/Africa
64,United Nations African Institute for Economic ...,http://www.unidep.org/,,Middle East/Africa
65,West African Development Bank,http://www.boad.org/,boadsiege@boad.org,Middle East/Africa
66,West African Power Pool,wapp/index.shtml,,Middle East/Africa
